In [1]:
import os 
os.chdir('../')

In [8]:
import orca
from urbansim_templates import modelmanager as mm
from urbansim_templates.models import LargeMultinomialLogitStep
from urbansim_templates.utils import get_data, get_df

from choicemodels.tools import MergedChoiceTable

from urbansim.models.util import columns_in_formula, apply_filter_query

In [10]:
region_code = '06197001'
orca.add_injectable('running_calibration_routine', False)
orca.add_injectable('local_simulation', True)
orca.add_injectable('initial_run', False)
orca.add_injectable('region_code', region_code)
orca.add_injectable('base_year', 2010)
orca.add_injectable('forecast_year', 2011)
orca.add_injectable('calibrated', True)
orca.add_injectable('calibrated_folder', 'custom')
orca.add_injectable('multi_level_lcms', True)
orca.add_injectable('segmented_lcms', True)
orca.add_injectable('capacity_boost', 1)
orca.add_injectable('all_local', True)
orca.add_injectable('skim_source', 'beam')

In [14]:
import datasources
import variables
import models

importing datasources
importing datasources for region 06197001
custom_mpo_06197001_model_data.h5
Checking if custom_settings.yaml file exists
Checking if custom output_parameters.yaml file exists


/Users/juandavidcaicedocastro/Dropbox/01_berkeley/22_UrbanSim/01_projects/MLCM/02_github/DEMOS_URBANSIM/demos_urbansim/datasources.py:356: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hct = hct.append(forecast_hct.reset_index())
/Users/juandavidcaicedocastro/Dropbox/01_berkeley/22_UrbanSim/01_projects/MLCM/02_github/DEMOS_URBANSIM/demos_urbansim/datasources.py:388: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ect = ect.append(forecast_ect.reset_index())


Output path exists!
importing variables for region 06197001


ModuleNotFoundError: No module named 'stopwatch'

In [2]:
orca.add_injectable('running_calibration_routine', False)
orca.add_injectable('local_simulation', True)
orca.add_injectable('initial_run', False)
orca.add_injectable('region_code', region_code)
orca.add_injectable('base_year', 2010)
orca.add_injectable('forecast_year', 2011)
orca.add_injectable('calibrated', True)
orca.add_injectable('calibrated_folder', 'custom')
orca.add_injectable('multi_level_lcms', True)
orca.add_injectable('segmented_lcms', True)
orca.add_injectable('capacity_boost', 1)
orca.add_injectable('all_local', True)
orca.add_injectable('skim_source', 'beam')

'/Users/juandavidcaicedocastro/Dropbox/01_berkeley/22_UrbanSim/01_projects/MLCM/02_github/DEMOS_URBANSIM/demos_urbansim'

['settings.yaml',
 'agetest.py',
 'marriage.py',
 'models.py',
 'requirements.txt',
 'variables.py',
 'plotting.py',
 'simulate.py',
 'indicators.py',
 '__pycache__',
 'checkHouses.py',
 'initr.py',
 'fatalityTest.py',
 'personsLength.py',
 'utils.py',
 'datasources.py',
 'configs',
 'process_skims.py',
 'yaml_notebook.ipynb',
 'data',
 'outputs',
 'wget-log',
 'runs',
 'notebooks']